In [9]:
from langchain_openai import AzureChatOpenAI

# Azure OpenAI Configuration
AZURE_OPENAI_ENDPOINT = "https://aishu-m8q3ed4m-swedencentral.cognitiveservices.azure.com/"
AZURE_OPENAI_KEY = "028qDuTdd4Z5y0nsdbVns8ZesZeQxt4NEQmW33BObOs7cLO9gIteJQQJ99BCACfhMk5XJ3w3AAAAACOGWE1L"
AZURE_DEPLOYMENT_NAME = "gpt-4o"

# Initialize the Azure Chat Model in LangChain
llm = AzureChatOpenAI(
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    openai_api_version="2025-01-01-preview",
    azure_deployment=AZURE_DEPLOYMENT_NAME,
    api_key=AZURE_OPENAI_KEY,
)

In [ ]:
# Step 1: Scene Breakdown using GPT-4o
def breakdown_scene(scene_description):
    system_prompt = "This is for education purpose. Break down the following scene into short, cinematic frame descriptions. There should be maximum 5 scenes and prefer lesser so as just to include the important details of the scene. Strictly use language of responsible AI to avoid content policy issues in image generation for example instead of gun use waster pistol, instead of blood use red liquid, etc. Strictly only output scenes of which image can be generated by dall-e 3 and all the scenes should be seperated by '///'. Tag consistent characters. Don't use words like Criminal or Victim"

    response = llm.invoke(f"{system_prompt}\n\n{scene_description}")
    content = response.content
    return content.split("\n")

resp = breakdown_scene("The victim was having lunch in a restaurant at night around 8pm by the street and the criminal came with a gun masked up and shot the victim.")

In [23]:
from openai import AzureOpenAI
import os
import requests
from PIL import Image
import json


client = AzureOpenAI(
    api_version="2025-01-01-preview",  
    api_key="30GVlMjbtF3pTphG8eJ6a7eXLJ3BbasyZZnRS1YRSLRR0qilthcYJQQJ99BCACfhMk5XJ3w3AAAAACOGj4uE",  
    azure_endpoint="https://nikhi-m8q67l88-swedencentral.openai.azure.com/"
)

In [ ]:
scenes = "".join(resp).split("///")

counter = 1

for scene in scenes:
    scene = scene.strip()
    print(scene)
    result = client.images.generate(
        model="dall-e-3",
        prompt="This is for educational purpose so replace objects which you feel are breaching policy:" + scene,
        n=1
    )

    json_response = json.loads(result.model_dump_json())

    # Set the directory for the stored image
    image_dir = os.path.join(os.curdir, 'video_images')

    # If the directory doesn't exist, create it
    if not os.path.isdir(image_dir):
        os.mkdir(image_dir)

    # Initialize the image path (note the filetype should be png)
    image_path = os.path.join(image_dir, f"generated_image_{counter}.png")

    # Retrieve the generated image
    image_url = json_response["data"][0]["url"]  # extract image URL from response
    generated_image = requests.get(image_url).content  # download the image
    with open(image_path, "wb") as image_file:
        image_file.write(generated_image)

    # Display the image in the default image viewer
    # image = Image.open(image_path)
    # image.show()
    print(image_path)

    counter += 1
    

Certainly! Here's the breakdown of the described scene using responsible and acceptable language that aligns with image-generation constraints:---**Scene 1:**  A restaurant by the street at 8 PM with warm glowing lights. A person (Tag: Civilian A) is seated outdoors at a table, calmly enjoying their meal under the night sky. Other patrons are seated nearby, creating a peaceful urban atmosphere.
./images/generated_image_1.png
**Scene 2:**  A masked individual (Tag: Civilian B) is approaching the restaurant from the street, carrying an object resembling a water pistol. Their face is concealed by a scarf or mask, and they appear focused on their direction.
./images/generated_image_2.png
**Scene 3:**  Close-up shot of Civilian A looking up in surprise, holding a spoon mid-air. Civilian B is now standing near them, aiming the water pistol in their direction. The tense atmosphere is reflected in body language and facial expressions.
./images/generated_image_3.png
**Scene 4:**  A dramatic mom

In [ ]:
from moviepy import ImageSequenceClip

fps_value = 5/28

clip = ImageSequenceClip(['video_images/generated_image_1.png', 'video_images/generated_image_2.png', 'video_images/generated_image_3.png', 'video_images/generated_image_4.png','video_images/generated_image_5.png'], fps = fps_value) 
  
# showing  clip  
clip.display_in_notebook()

In [13]:
speech_to_say = llm.invoke("This is For educational purpose. Summarize the scene in speech with tone of describing. The text should not result in audio output more than 30-40 secs. The victim was having lunch in a restaurant at night around 8pm by the street and the antagonist came masked up and dismissed the victim. JUST PURE TEXT NO EXPLANATIONS")

In [15]:
print(speech_to_say.content)

It was a quiet night around 8 PM, the street bathed in the soft glow of dim streetlights. Inside the modest restaurant, the victim sat alone, savoring a peaceful meal. Suddenly, a shadow entered—a masked figure, enshrouded in cold intent. Without hesitation, the antagonist approached swiftly, an air of menace filling the room, and in a chilling, calculated instant, dismissed the unsuspecting victim. The casual serenity of the night shattered, leaving behind only stillness and unanswered questions.


In [34]:
from moviepy import VideoFileClip
scene_clip = VideoFileClip("scene_enact.mp4")
avatar_clip = VideoFileClip("avatar_tts.webm").resized(0.25)


{'video_found': True, 'audio_found': False, 'metadata': {'major_brand': 'isom', 'minor_version': '512', 'compatible_brands': 'isomiso2avc1mp41', 'encoder': 'Lavf61.7.100'}, 'inputs': [{'streams': [{'input_number': 0, 'stream_number': 0, 'stream_type': 'video', 'language': None, 'default': True, 'size': [1024, 1024], 'bitrate': 381, 'fps': 0.18, 'codec_name': 'h264', 'profile': '(Constrained Baseline)', 'metadata': {'Metadata': '', 'handler_name': 'VideoHandler', 'vendor_id': '[0][0][0][0]', 'encoder': 'Lavc61.19.100 libx264'}}], 'input_number': 0}], 'duration': 27.78, 'bitrate': 381, 'start': 0.0, 'default_video_input_number': 0, 'default_video_stream_number': 0, 'video_codec_name': 'h264', 'video_profile': '(Constrained Baseline)', 'video_size': [1024, 1024], 'video_bitrate': 381, 'video_fps': 0.18, 'video_duration': 27.78, 'video_n_frames': 5}
/Users/nikhilmankani/Documents/SherlockAI/.env/lib/python3.12/site-packages/imageio_ffmpeg/binaries/ffmpeg-macos-x86_64-v7.1 -i scene_enact.mp

/Users/nikhilmankani/Documents/SherlockAI/.env/lib/python3.12/site-packages/moviepy/video/io/ffmpeg_reader.py:517: UserWarning: Subtitle stream parsing is not supported by moviepy and will be ignored
ffmpeg output:

Input #0, matroska,webm, from 'avatar_tts.webm':
  Metadata:
    ENCODER         : Lavf61.9.107
  Duration: 00:00:28.01, start: -0.007000, bitrate: 2717 kb/s
  Stream #0:0: Video: vp9 (Profile 0), yuv420p(tv, progressive), 1920x1080, SAR 1:1 DAR 16:9, 25 fps, 25 tbr, 1k tbn
      Metadata:
        alpha_mode      : 1
        ENCODER         : Lavc61.33.102 libvpx-vp9
        DURATION        : 00:00:28.000000000
  Stream #0:1: Audio: opus, 48000 Hz, mono, fltp
      Metadata:
        ENCODER         : Lavc61.33.102 libopus
        DURATION        : 00:00:28.008000000
  Stream #0:2: Subtitle: webvtt
      Metadata:
        ENCODER         : Lavc61.33.102 webvtt
        DURATION        : 00:00:27.900000000
At least one output file must be specified

  warnings.warn(


In [35]:
avatar_clip = avatar_clip.with_position(("right", "bottom"))

In [36]:
from moviepy import CompositeVideoClip
final_video = CompositeVideoClip([scene_clip, avatar_clip])
final_video.write_videofile("final_pip_output.mp4", codec="libx264")

                                                                         
                                                                         
frame_index:  33%|███▎      | 410/1250 [07:26<01:29,  9.38it/s, now=None]

MoviePy - Building video final_pip_output.mp4.
MoviePy - Writing audio in final_pip_outputTEMP_MPY_wvf_snd.mp3








                                                                         
                                                                         
frame_index:  33%|███▎      | 410/1250 [07:26<01:29,  9.38it/s, now=None]

MoviePy - Done.
MoviePy - Writing video final_pip_output.mp4



























































































































































































































































































































































































































































































































































































































































































































                                                                         
                                                                         
frame_index:  33%|███▎      | 410/1250 [08:43<01:29,  9.38it/s, now=None]

MoviePy - Done !
MoviePy - video ready final_pip_output.mp4
